# Using bandits to learn probabilities for the search space

To learn probabilities for the search space, the `Brush` estimators (not the DEAP!) uses a multi-armed bandit approach. 

This allows it to dynamically adjust the sampling probabilities of different operations based on their performance.

Not only it can learn how to sample terminals, but it also learns which mutations are useful and which are not. This is also important, as the mutations are responsible for sampling from the search space.

In this notebook, we show how to set this configuration, and how it changes the probabilities.

In [1]:
import pandas as pd
from pybrush import Dataset, SearchSpace
from pybrush import BrushRegressor

df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

data = Dataset(X,y)

search_space = SearchSpace(data)

This is the original search space using the loaded dataset. 

In [2]:
print( search_space.print() )

Search Space
===
terminal_map: {"ArrayB": ["1.00"], "ArrayI": ["x_5", "x_7", "1.00"], "ArrayF": ["x_0", "x_1", "x_2", "x_3", "x_4", "x_6", "1.00", "1.00*MeanLabel"]}
terminal_weights: {"ArrayB": [-nan], "ArrayI": [0.011619061, 0.03579926, 0.023709161], "ArrayF": [0.6343385, 0.67299956, 0.42711574, 0.8625447, 0.8957853, 0.20750472, 0.6167148, 0.6167148]}
node_map[ArrayI][["ArrayF", "ArrayI", "ArrayI"]][SplitOn] = SplitOn, weight = 1
node_map[ArrayI][["ArrayI", "ArrayI", "ArrayI"]][SplitOn] = SplitOn, weight = 1
node_map[ArrayI][["ArrayI", "ArrayI"]][SplitBest] = SplitBest, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Logistic] = Logistic, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Square] = Square, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Sqrtabs] = Sqrtabs, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Sqrt] = Sqrt, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "Ar

There are two important settings for this:

- `bandit`, which specifies which strategy will be used to learn the probabilities, and
- `weights_init`, which will initialize the sampling probabilities for terminals in the search space based on the correlation between each terminal and the target variable.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=100,
    objectives=["error", "complexity"],

    # Changing search space sampling probabilities
    bandit='thompson',
    weights_init=True,
    
    verbosity=1
)

In [4]:
est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Completed 100% [====================]
Saved population to file /tmp/tmpshx7uzc1/population.json
saving final population as archive...
score: 0.8900709673753779


The attribute `search_space_` is an original copy and it is not changed.

In [5]:
est.search_space_.print()

Search Space
===
terminal_map: {"ArrayB": ["1.00"], "ArrayI": ["x5", "x7", "1.00"], "ArrayF": ["x0", "x1", "x2", "x3", "x4", "x6", "1.00", "1.00*MeanLabel"]}
terminal_weights: {"ArrayB": [-nan], "ArrayI": [0.011619061, 0.03579926, 0.023709161], "ArrayF": [0.6343385, 0.67299956, 0.42711574, 0.8625447, 0.8957853, 0.20750472, 0.6167148, 0.6167148]}
node_map[ArrayI][["ArrayI", "ArrayI"]][SplitBest] = SplitBest, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Logabs] = Logabs, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Exp] = Exp, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Sin] = Sin, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Cos] = Cos, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Logabs] = Logabs, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Exp] = Exp, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Sin] = Sin, weight = 1
node_map[Matrix

When we call `fit`, the Brush engine is set to run and return the best individual. Then, the engine is stored as an attribute in our estimator, called `engine_`, and we can retrieve the learned probabilities by looking at the engine's search space.

In [6]:
est.engine_.search_space.print()

Search Space
===
terminal_map: {"ArrayB": ["1.00"], "ArrayI": ["x5", "x7", "1.00"], "ArrayF": ["x0", "x1", "x2", "x3", "x4", "x6", "1.00", "1.00*MeanLabel"]}
terminal_weights: {"ArrayB": [-nan], "ArrayI": [0.5, 0.5, 0.5], "ArrayF": [0.5302013, 0.5300546, 0.55172414, 0.55825245, 0.5, 0.55445546, 0.407767, 0.3604651]}
node_map[ArrayI][["ArrayI", "ArrayI"]][SplitBest] = SplitBest, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Logabs] = Logabs, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Exp] = Exp, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Sin] = Sin, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF", "ArrayF"]][Cos] = Cos, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Logabs] = Logabs, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Exp] = Exp, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "ArrayF"]][Sin] = Sin, weight = 1
node_map[MatrixF][["ArrayF", "ArrayF", "Array

Mutation probabilities are stored in the parameters, not the search space. The same way that we can have the original and the learned probabilities by looking at the instance references and the instance's engine reference, we can look the learned probabilities.

In [7]:
print(est.parameters_.cx_prob)
print(est.parameters_.mutation_probs)

0.1428571492433548
{'delete': 0.1666666716337204, 'insert': 0.1666666716337204, 'point': 0.1666666716337204, 'subtree': 0.1666666716337204, 'toggle_weight_off': 0.1666666716337204, 'toggle_weight_on': 0.1666666716337204}


In [8]:
print(est.engine_.params.cx_prob)
print(est.engine_.params.mutation_probs)

0.19159255921840668
{'delete': 0.3327217102050781, 'insert': 0.014492753893136978, 'point': 0.47653430700302124, 'subtree': 0.687393069267273, 'toggle_weight_off': 0.40665435791015625, 'toggle_weight_on': 0.019417475908994675}
